In [ ]:
// This top block may not need to be run at all, you can start with the using statements.
// using nightly-build
//#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json"
#r "nuget: Microsoft.Data.Analysis, 0.20.0-preview.22356.1"
#r "nuget: Microsoft.ML, 2.0.0-preview.22356.1"
#r "nuget:Microsoft.DotNet.Interactive.ExtensionLab,*-*"
#r "nuget:SandDance.InteractiveExtension, 1.0.53"
//#r "nuget: XPlot.Plotly.Interactive, 4.0.6"


In [ ]:
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;
using Microsoft.DotNet.Interactive.Formatting;
using Microsoft.DotNet.Interactive;
//using XPlot.Plotly.Interactive;
using Microsoft.Data.Analysis;
using Microsoft.ML;
using System.Collections.Generic;
using System.IO;
using System.Linq;

In [ ]:
var fileExists = File.Exists("draft-info/2024-04-23-top-schools.csv");
if (fileExists)
{
    display("This file exists.  Nothing has blown up.....yet.");
}
var nextDraft = DataFrame.LoadCsv("draft-info/2024-04-23-top-schools.csv");

In [ ]:
nextDraft.Columns.Add(new PrimitiveDataFrameColumn<int>("SuggestedAuctionValue", nextDraft.Rows.Count()));
nextDraft.Columns.Add(new StringDataFrameColumn("LeagifyPosition", nextDraft.Rows.Count()));

In [ ]:
var conferenceToPosition = DataFrame.LoadCsv("conferenceToPosition.csv");


for (int i = 0; i < nextDraft.Rows.Count; i++)
{
    switch((string)nextDraft.Columns["Conference"][i])
    {
        case "SEC":
            nextDraft.Columns["LeagifyPosition"][i] = "SEC";
            break;
        case "ACC":
            nextDraft.Columns["LeagifyPosition"][i] = "ACC";
            break;
        case "Big 12":
            nextDraft.Columns["LeagifyPosition"][i] = "Big 12";
            break;
        case "Pac 12":
            nextDraft.Columns["LeagifyPosition"][i] = "Pac 12";
            break;
        case "Big Ten":
            nextDraft.Columns["LeagifyPosition"][i] = "Big Ten";
            break;
        default:
            nextDraft.Columns["LeagifyPosition"][i] = "RandomSmallSchool";
            break;
    }
    if ((string)nextDraft.Columns["School"][i] == "Notre Dame")
    {
        nextDraft.Columns["LeagifyPosition"][i] = "Flex";
    }
}


nextDraft



In [ ]:
nextDraft.Columns.Add(new PrimitiveDataFrameColumn<double>("ProjectedPointsAboveAverage", nextDraft.Rows.Count()));
nextDraft.Columns.Add(new PrimitiveDataFrameColumn<double>("ProjectedPointsAboveReplacement", nextDraft.Rows.Count()));
nextDraft.Columns.Add(new PrimitiveDataFrameColumn<double>("AveragePointsForPosition", nextDraft.Rows.Count()));
nextDraft.Columns.Add(new PrimitiveDataFrameColumn<double>("ReplacementValueAverageForPosition", nextDraft.Rows.Count()));

nextDraft

In [ ]:
// Let's do some stats on available schools here.
// Get the distinct LeagifyPosition values
string[] leagifyPositions = {"ACC", "SEC", "Big Ten", "Big 12", "Pac 12", "RandomSmallSchool", "Flex"};

IDictionary<string, double> draftableAverages = new Dictionary<string, double>();
IDictionary<string, double> replacementAverages = new Dictionary<string, double>();

// Loop over each LeagifyPosition value and get the corresponding rows
foreach (var position in leagifyPositions)
{
    Console.WriteLine("Leagify Position: " + position);
    var rowsForPosition = nextDraft.Rows
        .Where(row => row[6].ToString() == position.ToString());
    
    // Make sure rows are in descending order by potential points.
    var sortedRows = rowsForPosition
    .OrderByDescending(row => row[2])
    .ToList();

    double averageOfTopRows;
    double averageOfRemainingRows;
    if(position.Equals("SEC") || position.Equals("Big Ten"))
    {
        display($"Conference {position} needs a top 10.");
        var topRows = sortedRows.Take(10);
        averageOfTopRows = topRows.Average(row => Convert.ToDouble(row[2]));
        display($"Average points for draftable schools from {position}: {averageOfTopRows}");
        var remainingRows = sortedRows;
        remainingRows.RemoveRange(0,10);
        averageOfRemainingRows = remainingRows.Average(row => Convert.ToDouble(row[2]));
        display($"Average points for replacement value schools from {position}: {averageOfRemainingRows}");


        
    }
    else if(position.Equals("Flex"))
    {
        display("This is Notre Dame, I'm not doing stats on this school.");
        averageOfTopRows = sortedRows.Average(row => Convert.ToDouble(row[2]));
        averageOfRemainingRows = sortedRows.Average(row => Convert.ToDouble(row[2]));
    }
    else
    {
        display($"Conference {position} only needs a top 5");
        var topRows = sortedRows.Take(5);
        averageOfTopRows = topRows.Average(row => Convert.ToDouble(row[2]));
        display($"Average points for draftable schools from {position}: {averageOfTopRows}");

        var remainingRows = sortedRows;
        remainingRows.RemoveRange(0,5);
        averageOfRemainingRows = remainingRows.Average(row => Convert.ToDouble(row[2]));
        display($"Average points for replacement value schools from {position}: {averageOfRemainingRows}");
    }

    draftableAverages.Add(position, Math.Truncate(averageOfTopRows * 100) / 100);
    replacementAverages.Add(position, Math.Truncate(averageOfRemainingRows * 100) / 100);
}





In [ ]:
display(draftableAverages);
display(replacementAverages);

// Iterate over the rows and set the value of the new column based on the value in the Name column
for (int i = 0; i < nextDraft.Rows.Count; i++)
{
    string leagifyPosition = "";
    switch((string)nextDraft.Columns["LeagifyPosition"][i])
    {
        
        case "SEC":
            leagifyPosition = "SEC";
            break;
        case "ACC":
            leagifyPosition = "ACC";
            break;
        case "Big 12":
            leagifyPosition = "Big 12";
            break;
        case "Pac 12":
            leagifyPosition = "Pac 12";
            break;
        case "Big Ten":
            leagifyPosition = "Big Ten";
            break;
        case "RandomSmallSchool":
            leagifyPosition = "RandomSmallSchool";
            break;
        case "Flex":
            leagifyPosition = "Flex";
            break;
        default:
            leagifyPosition = "RandomSmallSchool";
            break;

    }

    nextDraft.Columns["AveragePointsForPosition"][i] = draftableAverages[leagifyPosition];
    var projectedPointsAboveAverage = (Single)nextDraft.Columns["ProjectedPoints"][i] - draftableAverages[leagifyPosition];
    nextDraft.Columns["ProjectedPointsAboveAverage"][i] = Math.Truncate(projectedPointsAboveAverage * 100) / 100;

    nextDraft.Columns["ReplacementValueAverageForPosition"][i] = replacementAverages[leagifyPosition];
    var projectedPointsAboveReplacement = (Single)nextDraft.Columns["ProjectedPoints"][i] - replacementAverages[leagifyPosition];
    nextDraft.Columns["ProjectedPointsAboveReplacement"][i] = Math.Truncate(projectedPointsAboveReplacement * 100) / 100;
    display($"School {(string)nextDraft.Columns["School"][i]} in conference {(string)nextDraft.Columns["Conference"][i]} has Leagify position {leagifyPosition}");
}

display(nextDraft);

In [ ]:
//nextDraft.Columns.Remove("SchoolURL");
DataFrame.WriteCsv(nextDraft,"auctionDraftAverages-2024.csv",',');